# THERMOMETRY (PLOTTING AND ANALYSIS)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind


data = pd.read_csv("THERMOMETRY (MJNPs-FLA).csv")
data2 = pd.read_csv("THERMOMETRY (CONTROL).csv")

grouped_data = data.groupby('TIME')['TEMP'].agg(['mean', 'std']).reset_index()
grouped_data2 = data2.groupby('TIME')['TEMP'].agg(['mean', 'std']).reset_index()

time1 = grouped_data['TIME']
mean_temperature = grouped_data['mean']
std_dev_temperature = grouped_data['std']

time2 = grouped_data2['TIME']
mean_temperature2 = grouped_data2['mean']
std_dev_temperature2 = grouped_data2['std']

plt.plot(time1, mean_temperature, label='MJNPs-FLA', color='blue')
plt.plot(time2, mean_temperature2, label='Control', color='green')

plt.errorbar(time1, mean_temperature, yerr=std_dev_temperature, fmt='o', color='red')
plt.errorbar(time2, mean_temperature2, yerr=std_dev_temperature2, fmt='o', color='m')


celsius_symbol = '(\u00b0C)'
plt.xlabel('Time (min)')
plt.ylabel('Temperature' + (celsius_symbol))
plt.title('Linear Curve of Temperature against Time')
plt.legend()


for t in set(time1).intersection(set(time2)):
    temp1 = data[data['TIME'] == t]['TEMP']
    temp2 = data2[data2['TIME'] == t]['TEMP']
    p_value = ttest_ind(temp1, temp2)[1]
    
    if p_value < 0.001:
        plt.text(t, (mean_temperature[time1 == t] + mean_temperature2[time2 == t]) / 2, "***", fontsize=14, color='black', ha='center')
    elif p_value < 0.01:
        plt.text(t, (mean_temperature[time1 == t] + mean_temperature2[time2 == t]) / 2, "**", fontsize=14, color='black', ha='center')
    elif p_value < 0.05:
        plt.text(t, (mean_temperature[time1 == t] + mean_temperature2[time2 == t]) / 2, "*", fontsize=14, color='black', ha='center')

plt.grid(True)
plt.tight_layout()
plt.savefig('THERMOMETRY.tif', dpi=600, format='tif')
plt.show()

# CEM43 (PLOTTING AND ANALYSIS)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

def cem_43(ti, T):
    if T < 43:
        R = 1/4
    else:
        R = 1/2
    return ti * R**(43 - T)

csv_file = "THERMOMETRY (MJNPs-FLA).csv"
data = pd.read_csv(csv_file)
cem43_values = []

for index, row in data.iterrows():
    ti = row['TIME']
    T = row['TEMP']
    cem43_values.append(cem_43(ti, T))

data['CEM43'] = cem43_values
cem_data = data[['TIME', 'TEMP', 'CEM43']]

csv_file2 = "THERMOMETRY (CONTROL).csv"
data2 = pd.read_csv(csv_file2)
cem43_values2 = []

for index, row in data2.iterrows():
    ti = row['TIME']
    T = row['TEMP']
    cem43_values2.append(cem_43(ti, T))

data2['CEM43'] = cem43_values2
cem_data2 = data2[['TIME', 'TEMP', 'CEM43']]

celsius_symbol = '(\u00b0C)'
plt.scatter(cem_data['TEMP'], cem_data['CEM43'], color='blue', label='MJNPs-FLA', marker='p')
plt.scatter(cem_data2['TEMP'], cem_data2['CEM43'], color='orange', label='Control', marker='^')

plt.xlabel('Temperature' + celsius_symbol)
plt.ylabel('CEM43')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('CEM43.tif', dpi=600, format='tif')
plt.show()

time_10min_cem = cem_data[cem_data['TIME'] == 10]['CEM43'].iloc[0]
time_30min_cem = cem_data[cem_data['TIME'] == 30]['CEM43'].iloc[0]

time_10min_cem2 = cem_data2[cem_data2['TIME'] == 10]['CEM43'].iloc[0]
time_30min_cem2 = cem_data2[cem_data2['TIME'] == 30]['CEM43'].iloc[0]

print("CEM43 for 10 minutes (MJNPs-FLA):", time_10min_cem)
print("CEM43 for 30 minutes (MJNPs-FLA):", time_30min_cem)
print("CEM43 for 10 minutes (Control):", time_10min_cem2)
print("CEM43 for 30 minutes (Control):", time_30min_cem2)

t_statistic_10min, p_value_10min = ttest_ind(cem_data[cem_data['TIME'] == 10]['CEM43'], cem_data2[cem_data2['TIME'] == 10]['CEM43'])
t_statistic_30min, p_value_30min = ttest_ind(cem_data[cem_data['TIME'] == 30]['CEM43'], cem_data2[cem_data2['TIME'] == 30]['CEM43'])

print("P-value for time 10 minutes:", p_value_10min)
print("P-value for time 30 minutes:", p_value_30min)
